In [1]:
import pandas as pd
xl_data = pd.read_csv("csxl_data.csv")
app_lab_data = pd.read_csv("app-lab-data.csv")

In [ ]:
# Convert string to TimeDelta Object.
data['date'] = pd.to_datetime(data['date'])

data['timeIn'] = pd.to_timedelta(data['timeIn'])

data['start'] = data['date'] + data['timeIn']

data['day_of_week'] = data['start'].dt.day_name()

data['civilian_time'] = data['start'].dt.strftime("%I %p").str.lstrip("0")

reservations_per_hour = data.groupby(["day_of_week", "civilian_time"]).size().reset_index(name="count")

reservations_per_hour.head(5)